# Data restructuring


After webscraping the valuable information, the data needs to be transformed to have a proper dataset. Usually this step would have been done in the same file as the webscraping, but due to the impossibility of sharing the webscraping notebook for legal reasons, **this notebook helps in understanding the data retrieved from the website scraped.**


In this case, the data retrieved in a csv format has **4 columns**:

   - **index**
   - **Name:** Name of the offer. *Small description of the house offered: type of house (first word), principal feature and the location.* 
   - **Features:** List of features webscraped. *Different type of features for each observation (Nº of rooms, Surface, Nº of bathrooms, Elevator, ...).*
   - **Price:** Price of the rental.
 
After transforming the data, the new dataset downloaded has **12 columns**:

   - **House_type:** Type of the house written in the offer. *Retrieved from **Name***.
   - **Location:** District of the city where the house is located. *Retrieved from **Name***.
   - **Furnished:** Whether the house is furnished or not. *Retrieved from **Features***.
   - **Elevator:** Whether the house has an elevator or not. *Retrieved from **Features***.
   - **Terrace:** Whether the house has a terrace or not. *Retrieved from **Features***.
   - **Balcony:** Whether the house has a balcony or not. *Retrieved from **Features***.
   - **Storage:** Whether the house has a storage or not. *Retrieved from **Features***.
   - **Rooms:** Nº of rooms. *Retrieved from **Features***.
   - **Bathrooms:** Nº of bathrooms. *Retrieved from **Features***.
   - **Floor:** Nº of floor. *Retrieved from **Features***.
   - **Surface:** Square meters of the house. *Retrieved from **Features***.
   - **Price:** *Format fixed.*

In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import ast

In [3]:
df=pd.read_csv('rental_offers.csv',index_col='index')
df.head(4)

,Unnamed: 0,Name,Features,Price
index,,,,
0,0,"Piso de cuatro habitaciones Maximilià Thous, S...","['4 habitaciones', '125 m²', '2 baños', 'Plant...",€ 1.300/mes
1,1,"Piso de tres habitaciones del Timó, El Perello...","['3 habitaciones', '80 m²', '1 baño', 'Planta ...",€ 1.100/mes
2,2,"Piso de tres habitaciones del Timó, El Perello...","['3 habitaciones', '96 m²', '2 baños', 'Planta...",€ 1.195/mes
3,3,"Piso tercera planta, Sant Francesc, València","['5 habitaciones', '236 m²', '3 baños', 'Plant...",€ 4.200/mes


In [11]:
#Example: Observations 579 and 580 don't have the surface described in the offer
df[579:581]

,Unnamed: 0,Name,Features,Price
index,,,,
579,579,"Piso de dos habitaciones C. Cerrajeros, El Mer...","['2 habitaciones', '2 baños', 'Ascensor', 'Ter...",€ 1.875/mes
580,580,Piso de una habitación Carrer de Guillem de Ca...,"['1 habitación', '1 baño', 'Ascensor', 'Terraz...",€ 1.300/mes


In [3]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df.head(4)

,Name,Features,Price
index,,,
0,"Piso de cuatro habitaciones Maximilià Thous, S...","['4 habitaciones', '125 m²', '2 baños', 'Plant...",€ 1.300/mes
1,"Piso de tres habitaciones del Timó, El Perello...","['3 habitaciones', '80 m²', '1 baño', 'Planta ...",€ 1.100/mes
2,"Piso de tres habitaciones del Timó, El Perello...","['3 habitaciones', '96 m²', '2 baños', 'Planta...",€ 1.195/mes
3,"Piso tercera planta, Sant Francesc, València","['5 habitaciones', '236 m²', '3 baños', 'Plant...",€ 4.200/mes


In [4]:
# Drop duplicated offers
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)
print(df.duplicated().sum())


44
0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 755
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Name      712 non-null    object
 1   Features  712 non-null    object
 2   Price     712 non-null    object
dtypes: object(3)
memory usage: 22.2+ KB


In [6]:
df['House_type']=df['Name'].apply(lambda x: x.split(' ')[0])
df['House_type'].replace({'Dúplex,':'Dúplex'},inplace=True)
df['House_type'].value_counts()

House_type
Piso        690
Ático        10
Estudio       3
Casa          3
Edificio      3
Dúplex        2
Chalet        1
Name: count, dtype: int64

In [7]:
df['Location']=df['Name'].apply(lambda x: x.split(',')[-2].strip())
print(len(df['Location'].unique()))
df['Location'].value_counts()


72


Location
El Cabanyal-El Canyamelar    62
Russafa                      48
Sant Francesc                47
Arrancapins                  31
El Mercat                    29
                             ..
Favara                        1
Trinitat                      1
Ciutat Universitària          1
Vara de Quart                 1
Benimàmet                     1
Name: count, Length: 72, dtype: int64

In [8]:
print(type(df['Features'][0]))
df['Features2']=df['Features'].apply(lambda x: ast.literal_eval(x))
print(type(df['Features2'][0]))

<class 'str'>
<class 'list'>


In [9]:
#Check for all the possible features aggregating the lists of features for each observation and applying a Counter.
#Then define the features and their types to build new columns.
all_feats=sum(df['Features2'],[])
Counter(all_feats)
#Amueblado [bin], Ascensor (No)[bin], Terraza[bin], Balcón[bin], Trastero[bin] 
#Nº Habitaciones[int], Nº baños[int], Planta[int], Superficie (m2)[int]

Counter({'Amueblado': 538,
         'Ascensor': 499,
         '1 baño': 335,
         '2 baños': 316,
         'Terraza': 315,
         '2 habitaciones': 208,
         '3 habitaciones': 206,
         'Balcón': 167,
         'No Ascensor': 157,
         '4 habitaciones': 117,
         '1 habitación': 103,
         'Planta 3': 65,
         'Planta 1': 62,
         'Planta 2': 52,
         'Sólo cocina amueblada': 45,
         'Planta 4': 39,
         '80 m²': 32,
         'Trastero': 31,
         'Planta 5': 28,
         '90 m²': 22,
         '3 baños': 21,
         '110 m²': 20,
         '70 m²': 20,
         '100 m²': 19,
         'Planta 6': 19,
         '120 m²': 19,
         '60 m²': 17,
         '95 m²': 16,
         'Planta 7': 16,
         '5 habitaciones': 15,
         '75 m²': 15,
         '85 m²': 15,
         '65 m²': 13,
         '130 m²': 12,
         '50 m²': 12,
         'Planta 8': 10,
         'Planta 9': 10,
         '35 m²': 9,
         '45 m²': 9,
         '55 m²': 8

In [10]:
def furnished(x):
    if 'Amueblado' in x:
        return 1
    elif 'Parcialmente amueblado' in x:
        return 1
    else:
        return 0

df['Furnished']=df['Features2'].apply(lambda x: furnished(x))
df['Furnished'].value_counts()

Furnished
1    539
0    173
Name: count, dtype: int64

In [11]:
def elevator(x):
    if 'Ascensor' in x:
        return 1
    elif 'No Ascensor' in x:
        return 0
    else:
        return np.nan

df['Elevator']=df['Features2'].apply(lambda x: elevator(x))
df['Elevator'].value_counts(dropna=False)

Elevator
1.0    499
0.0    157
NaN     56
Name: count, dtype: int64

In [12]:
def terrace(x):
    if 'Terraza' in x:
        return 1
    else:
        return 0
def balcony(x):
    if 'Balcón' in x:
        return 1
    else:
        return 0

df['Terrace']=df['Features2'].apply(lambda x: terrace(x))
df['Balcony']=df['Features2'].apply(lambda x: balcony(x))
print(df['Terrace'].value_counts())
print(df['Balcony'].value_counts())

Terrace
0    397
1    315
Name: count, dtype: int64
Balcony
0    545
1    167
Name: count, dtype: int64


In [13]:
def storage(x):
    if 'Trastero' in x:
        return 1
    else:
        return 0
df['Storage']=df['Features2'].apply(lambda x: storage(x))
df['Storage'].value_counts()

Storage
0    681
1     31
Name: count, dtype: int64

In [14]:
def rooms(x):
    for i in x:
        if 'habitaci' in i:
            return i.split()[0]
    return np.nan
df['Rooms']=df['Features2'].apply(lambda x: rooms(x))
df['Rooms'].value_counts(dropna=False)

Rooms
2      208
3      206
4      117
1      103
NaN     59
5       15
5+       4
Name: count, dtype: int64

In [15]:
def bathrooms(x):
    for i in x:
        if 'baño' in i:
            return i.split()[0]
    return np.nan
df['Bathrooms']=df['Features2'].apply(lambda x: bathrooms(x))
df['Bathrooms'].value_counts(dropna=False)

Bathrooms
1      335
2      316
NaN     33
3       21
3+       7
Name: count, dtype: int64

In [16]:
def floor(x):
    for i in x:
        if 'Planta' in i:
            return i.split()[1]
    return np.nan
df['Floor']=df['Features2'].apply(lambda x: floor(x))
df['Floor'].value_counts(dropna=False)

Floor
NaN    379
3       65
1       62
2       52
4       39
5       28
6       19
7       16
8       10
9       10
10       8
11       7
B        6
19       3
21       2
12       2
15       1
18       1
13       1
17       1
Name: count, dtype: int64

In [17]:
def surface(x):
    for i in x:
        if 'm²' in i:
            return i.split()[0]
    return np.nan
df['Surface']=df['Features2'].apply(lambda x: surface(x))
df['Surface'].value_counts(dropna=False)

Surface
NaN    195
80      32
90      22
70      20
110     20
      ... 
27       1
123      1
61       1
188      1
57       1
Name: count, Length: 127, dtype: int64

In [18]:
df['Price2']=df['Price'].apply(lambda x: x.split(' ')[1].split('/')[0].replace('.',''))
df['Price2'].unique()

array(['1300', '1100', '1195', '4200', '1600', '2100', '2000', '1350',
       '1950', '1200', '1450', '1500', '1400', '1800', '2900', '950',
       '1010', '3800', '1050', '1130', '480', '450', '380', '1650', '300',
       '1850', '2700', '1980', '4000', '5500', '1630', '2250', '2050',
       '1370', '1290', '1835', '850', '1550', '1900', '1700', '2500',
       '1250', '1390', '1590', '1499', '1190', '949', '3050', '1490',
       '2400', '2600', '2200', '1680', '3000', '1240', '3600', '750',
       '900', '4800', '990', '1320', '1952', '1599', '1750', '1855',
       '1790', '1690', '1000', '5000', '1530', '1299', '2213', '1692',
       '3500', '3400', '1150', '2330', '2300', '1868', '200', '2650',
       '1301', '1020', '1090', '800', '1375', '1710', '1820', '970',
       '920', '820', '700', '1322', '1399', '1990', '3650', '4500',
       '2550', '2850', '2875', '3350', '2800', '980', '2440', '1875',
       '550', '22000', '1495', '2150', '3200', 'a', '4400', '2950',
       '3300', '80

In [19]:
df.loc[df.Price2=='a']

,Name,Features,Price,House_type,Location,Features2,Furnished,Elevator,Terrace,Balcony,Storage,Rooms,Bathrooms,Floor,Surface,Price2
index,,,,,,,,,,,,,,,,
664,"Piso de una habitación primera planta, El Caba...","['1 habitación', '2 baños', 'Planta 1', 'No As...",Precio a consultar,Piso,El Cabanyal-El Canyamelar,"[1 habitación, 2 baños, Planta 1, No Ascensor,...",1,0.0,0,1,0,1,2,1,NaN,a


In [20]:
df.drop(664,inplace=True)
df.loc[df.Price2=='a']

,Name,Features,Price,House_type,Location,Features2,Furnished,Elevator,Terrace,Balcony,Storage,Rooms,Bathrooms,Floor,Surface,Price2
index,,,,,,,,,,,,,,,,


In [21]:
df['Price2']=df['Price'].apply(lambda x: int(x.split(' ')[1].split('/')[0].replace('.','')))
df['Price2'].unique()

array([ 1300,  1100,  1195,  4200,  1600,  2100,  2000,  1350,  1950,
        1200,  1450,  1500,  1400,  1800,  2900,   950,  1010,  3800,
        1050,  1130,   480,   450,   380,  1650,   300,  1850,  2700,
        1980,  4000,  5500,  1630,  2250,  2050,  1370,  1290,  1835,
         850,  1550,  1900,  1700,  2500,  1250,  1390,  1590,  1499,
        1190,   949,  3050,  1490,  2400,  2600,  2200,  1680,  3000,
        1240,  3600,   750,   900,  4800,   990,  1320,  1952,  1599,
        1750,  1855,  1790,  1690,  1000,  5000,  1530,  1299,  2213,
        1692,  3500,  3400,  1150,  2330,  2300,  1868,   200,  2650,
        1301,  1020,  1090,   800,  1375,  1710,  1820,   970,   920,
         820,   700,  1322,  1399,  1990,  3650,  4500,  2550,  2850,
        2875,  3350,  2800,   980,  2440,  1875,   550, 22000,  1495,
        2150,  3200,  4400,  2950,  3300,  8000,  1270,  1385,  1030,
         835,   940,  1475,   500], dtype=int64)

In [22]:
df.drop(['Name','Features','Price','Features2'],axis=1,inplace=True)
df.rename({'Price2':'Price'},axis=1,inplace=True)
df.head()

,House_type,Location,Furnished,Elevator,Terrace,Balcony,Storage,Rooms,Bathrooms,Floor,Surface,Price
index,,,,,,,,,,,,
0,Piso,Sant Antoni,0,1.0,0,0,0,4,2,1,125,1300
1,Piso,El Perellonet,0,1.0,1,0,0,3,1,1,80,1100
2,Piso,El Perellonet,0,1.0,0,1,0,3,2,2,96,1195
3,Piso,Sant Francesc,0,1.0,0,0,0,5,3,3,236,4200
4,Piso,Gran Via,1,1.0,0,0,0,2,1,4,79,1600


In [23]:
df.to_csv('rental_offers_clean.csv')